In [1]:
pip install opencv-python PyQt5 pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import random
import threading
import time
import cv2
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout, QPushButton, QTextEdit, QGraphicsOpacityEffect
from PyQt5.QtGui import QImage, QPixmap, QIcon
from PyQt5.QtCore import QSize, QTimer, Qt, QPropertyAnimation, QPoint, QSequentialAnimationGroup
from PIL import Image
import numpy as np

# Function to handle camera access
def start_camera(camera_id=0):
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"Error: Could not open the camera with ID {camera_id}.")
        return
    return cap

# Convert OpenCV image to QImage for PyQt5
def convert_cv_qt(cv_img):
    rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    h, w, ch = rgb_image.shape
    bytes_per_line = ch * w
    convert_to_Qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
    return convert_to_Qt_format

# PyQt5 Application to show live feed with Instagram-like overlay
class InstagramLiveApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
        # Start the camera
        self.cap = start_camera()
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(20)

    def initUI(self):
        self.setWindowTitle("Instagram Live Simulation")
        self.setGeometry(100, 100, 720, 1280)
        self.setStyleSheet("background-color: black;")

        # Layout will be absolute positioning, not stacking layouts
        self.setLayout(QVBoxLayout())

        # Video feed display
        self.video_label = QLabel(self)
        self.video_label.setGeometry(0, 0, 720, 1280)  # Full-screen video

        # Add top bar with profile picture, live badge, and viewer count
        self.live_label = QLabel("LIVE", self)
        self.live_label.setGeometry(70, 30, 100, 40)  # Positioned at the top
        self.live_label.setStyleSheet("color: red; font-size: 24px; font-weight: bold; background: transparent;")

        self.viewers_label = QLabel("1.2K", self)
        self.viewers_label.setGeometry(200, 30, 100, 40)
        self.viewers_label.setStyleSheet("color: white; font-size: 20px; background: transparent;")

        self.profile_pic = QLabel(self)
        self.profile_pic.setPixmap(QPixmap("profile_pic.png").scaled(40, 40, Qt.KeepAspectRatio, Qt.SmoothTransformation))
        self.profile_pic.setGeometry(10, 25, 40, 40)
        self.profile_pic.setStyleSheet("background: transparent;")

        # Floating comments section (overlaid)
        self.comment_area = QTextEdit(self)
        self.comment_area.setGeometry(10, 900, 700, 200)  # Positioned at the bottom
        self.comment_area.setStyleSheet("background: rgba(0, 0, 0, 0.5); color: white; font-size: 18px; border: none;")
        self.comment_area.setReadOnly(True)

        # Add hearts (like button simulation)
        self.like_button = QPushButton(self)
        self.like_button.setIcon(QIcon("heart_icon.png"))
        self.like_button.setIconSize(QSize(40, 40))
        self.like_button.setGeometry(600, 1150, 50, 50)  # Bottom-right corner
        self.like_button.setStyleSheet("background: transparent;")
        self.like_button.clicked.connect(self.animate_heart)

        # Start threads for generating random comments and fluctuating viewers
        threading.Thread(target=self.generate_comments).start()
        threading.Thread(target=self.fluctuate_viewers).start()

    def update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            # Convert the frame to QImage and display it
            qt_img = convert_cv_qt(frame)
            self.video_label.setPixmap(QPixmap.fromImage(qt_img))

    def fluctuate_viewers(self):
        while True:
            fluctuation = random.randint(-500, 5000)
            viewers = max(0, random.randint(90000, 100000) + fluctuation)
            self.viewers_label.setText(f"{viewers} watching")
            time.sleep(2)

    def generate_comments(self):
        sample_comments = [
            "🔥 This is lit!", "No cap, this is amazing!", "Sheeesh! 😲", "You’re killin' it!",
            "Vibes on point! ✨", "Can't stop watching 😍", "Lowkey obsessed with this 🔥", "Yaaas! 🙌",
            "OMG I love this! 💖", "Fr tho, this is 🔥", "Bruh, how are you this good?!", "Iconic! ✨"
        ]
        while True:
            comment = random.choice(sample_comments)
            self.comment_area.append(f"User: {comment}")
            time.sleep(random.uniform(2, 5))

    def animate_heart(self):
        heart_label = QLabel(self)
        heart_label.setPixmap(QPixmap("heart_icon.png").scaled(40, 40, Qt.KeepAspectRatio, Qt.SmoothTransformation))
        heart_label.setGeometry(600, 1100, 40, 40)  # Start position (bottom-right)
        heart_label.setStyleSheet("background: transparent;")
        
        # Create animation to make the heart "float" up
        animation = QPropertyAnimation(heart_label, b"pos")
        animation.setDuration(2000)
        animation.setStartValue(QPoint(600, 1100))
        animation.setEndValue(QPoint(600, 700))  # End position (higher on the screen)

        # Create opacity effect to fade out
        opacity_effect = QGraphicsOpacityEffect(heart_label)
        heart_label.setGraphicsEffect(opacity_effect)
        fade_animation = QPropertyAnimation(opacity_effect, b"opacity")
        fade_animation.setDuration(2000)
        fade_animation.setStartValue(1)
        fade_animation.setEndValue(0)

        # Combine animations
        animation_group = QSequentialAnimationGroup()
        animation_group.addAnimation(animation)
        animation_group.addAnimation(fade_animation)
        animation_group.start()

        # Automatically remove heart after animation finishes
        animation_group.finished.connect(heart_label.deleteLater)

# Main Application
def main():
    app = QApplication(sys.argv)
    liveStream = InstagramLiveApp()
    liveStream.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


SystemExit: 0

C:\Users\brian\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
